In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import numpy as np

<div class="alert alert-success">    
    <h1>Problema 1: Assignació de tasques</h1>
    <br>
    Sigui $A$ una matriu de nombres enters de mida $n\times n$:
    $$A = \begin{pmatrix}a_{0,0}\quad \cdots \quad a_{0,n}\\
                         \vdots\quad\quad\quad\quad\vdots\\
                         a_{n,0}\quad \cdots \quad a_{n,n}
         \end{pmatrix}$$
    L'element $a_{i,j}$ correspon al cost d'assignar la tasca $i$ a l'empresa $j$.<br>
    Volem trobar el mínim cost d'assignar tasques a empreses amb la condició que totes les tasques han d'estar assignades i les empreses han de fer només una tasca.
     
</div>

## Funcions útils

In [3]:
costs = np.array([[11,12,18,40],
                  [14,15,13,22],
                  [11,17,19,23],
                  [17,14,20,28]])

####  <u>np.min</u>
Ens permet obtenir el mínim de cada columna o fila d'una matriu

In [4]:
print(costs.min(axis=0)) # Axis=0 indica que volem el mínim de cada columna
print(sum(costs.min(axis=0)))

[11 12 13 22]
58


In [5]:
print(costs.min(axis=1)) # Axis=1 indica que volem el mínim de cada fila
print(sum(costs.min(axis=1)))

[11 13 11 14]
49


#### <u>np.delete</u>
Ens permet eliminar una fila o una columna d'un array de numpy.<br>
Observa els exemples:

In [6]:
# Axis=0 indica que volem eliminar files. En aquest cas estem eliminant només la fila 2
new_costs1 = np.delete(costs, 2, axis=0) 
print(new_costs1)

[[11 12 18 40]
 [14 15 13 22]
 [17 14 20 28]]


In [7]:
# Axis=0 indica que volem eliminar files. En aquest cas estem eliminant les files 0 i 1
new_costs2 = np.delete(costs, [0,1], axis=0) 
print(new_costs2)

[[11 17 19 23]
 [17 14 20 28]]


In [8]:
# Axis=1 indica que volem eliminar columnes. En aquest cas estem eliminant les columnes 0 i 2
new_costs3 = np.delete(costs, [0,2], axis=1)
print(new_costs3)

[[12 40]
 [15 22]
 [17 23]
 [14 28]]


In [9]:
# Podem esborrar files i columnes d'una matriu
new_costs4 = np.delete(np.delete(costs, [0,1], axis=0), [2,3], axis=1)
print(new_costs4)

[[11 17]
 [17 14]]


In [10]:
from queue import PriorityQueue

def inf_bound(matrix):
    """
    Calcula la suma del mínim de cada columna
    
    Params
    ======
    :matrix: La matriu de costs
    
    Returns
    =======
    :inf: La suma del mínim de cada columna
    """
    
    # Mínim de cada columna
    if len(matrix)==0:
        return 0
    return sum(matrix.min(axis=0))

def sup_bound(matrix):
    """
    Retorna el cost d'una assignació qualsevol.
    
    Params
    ======
    :matrix: La matriu de costs
    
    Returns
    =======
    :sup: El cost d'una assignació qualsevol. Per exemple, podem retornar la suma de la diagonal de la matriu
          que consisteix en assignar la tasca 'i' a l'empresa 'i' on i=0,1,2,3,4...
    """
    
    # Assignació qualsevol. En aquest cas sumem la diagonal que consisteix a assignar
    # la tasca 'i' a l'empresa 'i' on i=0,1,2...
    return sum(matrix.diagonal()) if len(matrix)!=0 else 0

def tasks(matrix):
    """
    Troba l'assignació entre tasques i empreses amb cost mínim utilitzant ramificació i poda.
    Cada cop que troba una assignació millor ll'imprimeix per pantalla.
    
    Params
    =====
    :matrix: La matriu de costs
    """
    
    # Cotes inicials
    sup = sup_bound(matrix)
    inf = inf_bound(matrix)
    
    # Cua de prioritat. Guardarem quatre elements:
    # 1. Prioritat
    # 2. Parelles ja assignades (tasca, empresa)
    # 3. Tasca que hem d'assignar a continuació (row)
    # 4. Empreses ja assignades (col)
    pq = PriorityQueue()
    pq.put((inf, [], 0, set([])))
    
    # Iterarem mentre la cua de prioritat no sigui buida
    while not pq.empty():
        
        # Extraiem un element
        elem_cota, elem_list, elem_row, elem_cols = pq.get()                
            
        # Podem assignar la tasca 'elem_row' a qualsevol empresa 'col' que no haguem assignat encara
        for col in range(len(matrix)):
            if col not in elem_cols:
                
                # Copiem els originals ja que els modificarem
                new_elem_list, new_elem_cols = elem_list.copy(), elem_cols.copy()
                
                # Afegim els nous elements a la llista de visitats i afegim la parella
                new_elem_cols.add(col)                # Afegim l'empresa seleccionada
                new_elem_list.append((elem_row, col)) # Afegim una nova parella
                new_elem_row = elem_row + 1           # Indiquem que haurem de continuar amb una nova tasca
                
                # OPCIONAL: Ep! Si hem assignat la penúltima, l'última ja ens ve determinada
                if len(new_elem_list) == len(matrix)-1:
                    
                    # La tasca (erow) serà l'última (longitud de la matriu - 1)
                    # L'empresa (ecol) serà la que no estigui dins el conjunt d'empreses assignades
                    erow, ecol = len(matrix)-1, list(set(range(len(matrix))) - new_elem_cols)[0]                    
                    new_elem_cols.add(ecol)
                    new_elem_list.append((erow, ecol))
                    new_elem_row += 1
                
                # Com fem per calcular la cota?
                # Eliminem de la matriu les files i columnes que ja haguem usat
                matrix_slice = np.delete(matrix, list(range(0,new_elem_row)), 0)  # Files
                matrix_slice = np.delete(matrix_slice, list(new_elem_cols), 1)    # Columnes                
                
                # Calculem la cota amb la suma de parelles assignades + el mínim de la matriu restant
                new_elem_cota = sum(matrix[i,j] for i,j in new_elem_list) + inf_bound(matrix_slice)
                
                # Mirem si és solució. Això passarà si tenim les mateixes parelles dins la llista 
                # que la mida de la matriu
                if len(new_elem_list) == len(matrix):
                    
                    # En cas que haguem trobat una cota millor, imprimim i actualitzem la cota
                    if new_elem_cota < sup:                        
                        print("Millor solució trobada, cost:", new_elem_cota)  
                        print("Actualitzem la cota superior de", sup, "a", new_elem_cota)
                        
                        sup = new_elem_cota
                        
                        print("Assignacions: ")
                        for i, j in new_elem_list:
                            print('Tasca',i,'-> Empresa',j)
                        print("-"*60)
                
                # Altrament, només l'afegim si potencialment ens millora la cota
                elif new_elem_cota < sup:
                    pq.put((new_elem_cota, new_elem_list, new_elem_row, new_elem_cols))

In [11]:
costs = np.array([[11,12,18,40],
                  [14,15,13,22],
                  [11,17,19,23],
                  [17,14,20,28]])

print("Matriu de costs:")
print(costs)
print()

tasks(costs)

Matriu de costs:
[[11 12 18 40]
 [14 15 13 22]
 [11 17 19 23]
 [17 14 20 28]]

Millor solució trobada, cost: 64
Actualitzem la cota superior de 73 a 64
Assignacions: 
Tasca 0 -> Empresa 1
Tasca 1 -> Empresa 2
Tasca 2 -> Empresa 0
Tasca 3 -> Empresa 3
------------------------------------------------------------
Millor solució trobada, cost: 61
Actualitzem la cota superior de 64 a 61
Assignacions: 
Tasca 0 -> Empresa 0
Tasca 1 -> Empresa 2
Tasca 2 -> Empresa 3
Tasca 3 -> Empresa 1
------------------------------------------------------------


In [12]:
costs = np.array( [[38, 43, 49, 21, 25, 26, 18, 49],
                   [32, 48, 34, 38, 29, 16, 45, 44],
                   [45, 16, 29, 25, 39, 29, 32, 34],
                   [44, 30, 41, 36, 27, 34, 33, 24],
                   [34, 43, 39, 10, 23, 17, 39, 23],
                   [26, 28, 36, 45, 27, 47, 36, 45],
                   [28, 22, 42, 10, 38, 19, 38, 25],
                   [19, 36, 21, 46, 13, 39, 30, 24]])

print("Matriu de costs:")
print(costs)
print()

tasks(costs)

Matriu de costs:
[[38 43 49 21 25 26 18 49]
 [32 48 34 38 29 16 45 44]
 [45 16 29 25 39 29 32 34]
 [44 30 41 36 27 34 33 24]
 [34 43 39 10 23 17 39 23]
 [26 28 36 45 27 47 36 45]
 [28 22 42 10 38 19 38 25]
 [19 36 21 46 13 39 30 24]]

Millor solució trobada, cost: 165
Actualitzem la cota superior de 283 a 165
Assignacions: 
Tasca 0 -> Empresa 6
Tasca 1 -> Empresa 5
Tasca 2 -> Empresa 1
Tasca 3 -> Empresa 7
Tasca 4 -> Empresa 3
Tasca 5 -> Empresa 0
Tasca 6 -> Empresa 2
Tasca 7 -> Empresa 4
------------------------------------------------------------
Millor solució trobada, cost: 160
Actualitzem la cota superior de 165 a 160
Assignacions: 
Tasca 0 -> Empresa 6
Tasca 1 -> Empresa 5
Tasca 2 -> Empresa 1
Tasca 3 -> Empresa 7
Tasca 4 -> Empresa 3
Tasca 5 -> Empresa 4
Tasca 6 -> Empresa 0
Tasca 7 -> Empresa 2
------------------------------------------------------------
Millor solució trobada, cost: 154
Actualitzem la cota superior de 160 a 154
Assignacions: 
Tasca 0 -> Empresa 6
Tasca 1 -> Em

In [13]:
costs = np.random.randint(10,50,(10,10))

print("Matriu de costs:")
print(costs)
print()

tasks(costs)

Matriu de costs:
[[11 26 28 46 49 18 44 28 28 15]
 [38 33 26 19 47 17 31 30 11 17]
 [25 46 41 37 20 31 18 26 17 47]
 [46 47 21 47 13 16 18 41 44 28]
 [13 47 34 36 30 25 32 35 18 26]
 [48 12 32 44 47 48 42 22 41 24]
 [40 44 41 15 39 28 16 26 17 20]
 [22 39 19 30 39 41 45 19 21 44]
 [12 27 33 10 49 30 46 11 12 18]
 [24 34 38 25 30 36 49 36 16 36]]

Millor solució trobada, cost: 168
Actualitzem la cota superior de 293 a 168
Assignacions: 
Tasca 0 -> Empresa 9
Tasca 1 -> Empresa 5
Tasca 2 -> Empresa 8
Tasca 3 -> Empresa 4
Tasca 4 -> Empresa 0
Tasca 5 -> Empresa 1
Tasca 6 -> Empresa 6
Tasca 7 -> Empresa 2
Tasca 8 -> Empresa 3
Tasca 9 -> Empresa 7
------------------------------------------------------------
Millor solució trobada, cost: 158
Actualitzem la cota superior de 168 a 158
Assignacions: 
Tasca 0 -> Empresa 9
Tasca 1 -> Empresa 5
Tasca 2 -> Empresa 8
Tasca 3 -> Empresa 4
Tasca 4 -> Empresa 0
Tasca 5 -> Empresa 1
Tasca 6 -> Empresa 6
Tasca 7 -> Empresa 2
Tasca 8 -> Empresa 7
Tasca 9 -

<div class="alert alert-success">
    <h1>Problema 2: Sliding-Puzzle</h1>
    <br>
    Donat un taulell de $n\times n$ de nombres des d'$1$ fins a $n^2-1$ inicialment desordenats, volem trobar el nombre mínim de moviments possibles de manera que els nombres estiguin ordenats en ordre creixent i la casella sense número estigui a l'última posició.
</div>

<img src="https://www.researchgate.net/profile/Ruo-Ando/publication/347300656/figure/fig1/AS:969204928901121@1608087870493/Initial-state-and-goal-state-of-8-puzzle.ppm" width='25%'/>

In [37]:
from npuzzle import NPuzzle

# Inicialitzem un tauler i el barregem
board = NPuzzle()
board.create_board(n=3, moves=100)

print ("Un tauler aleatori:")
print(board)

Un tauler aleatori:
+-----------+
|   | 2 | 5 | 
+-----------+
| 1 | 3 | 4 | 
+-----------+
| 8 | 6 | 7 | 
+-----------+



La funció ``get_state_id`` ens retorna un 'string' amb la configuració del tauler. D'aquesta forma podrem guardar els estats que ja hem visitat per a no repetir-los


In [38]:
board.get_state_id()

'0,2,5,1,3,4,8,6,7'

Per a cada tauler, podem definir una cota que depèn del nombre de moviments que hem fet fins el moment i un valor optimista calculat com una 'distància' entre el tauler que estem considerant i el tauler objectiu.

$$C(X) = g(X) + h(X)$$
+ $g(X)$ és el nombre de passos que portem fins el moment.
+ $h(X)$ pot ser:
    + $h_1(X)$: El nombre de caselles que no estan al seu lloc sense tenir en compte la casella buida (hamming_distance)
    + $h_2(X)$: La suma de les distàncies de manhattan de cada casella al seu lloc correcte (manhattan_distance)

In [39]:
board.hamming_distance() # h1(X)

7

In [40]:
board.manhattan_distance() # h2(X)

12

Podem demanar quins moviments són valids des d'una configuració del tauler amb la funció ``allowed_moves()``. Un moviment consisteix en 'moure' la casella buida en una de les quatre direccions permeses:<br>
+ $L$: Left
+ $R$: Right
+ $U$: Up
+ $D$: Down

In [41]:
am = board.allowed_moves()
print(am)

['R', 'D']


Executem un moviment amb la funció ``move()``

In [43]:
new_board = board.move(am[0])
print(new_board)

+-----------+
| 2 |   | 5 | 
+-----------+
| 1 | 3 | 4 | 
+-----------+
| 8 | 6 | 7 | 
+-----------+



La funció state ens comprova si el nostre estat és un estat solució.

In [44]:
# Solucionat: True
# No solucionat: False
new_board.state()

False

In [45]:
import numpy as np

def solve_puzzle(board):
    """
    Soluciona el problema del N-Puzzle
    
    Params
    ======
    :board: Un objecte de la classe NPuzzle
    
    Returns
    =======
    :best_bound: Nombre de passos mínims per transformar el tauler d'entrada en el tauler objectiu
    :best_board: El tauler objectiu. Haurien de ser els números ordenats de petit a gran amb la casella buida al final.
    :expanded: El nombre de taulers expandits. Cada cop que traiem un tauler de la cua de prioritat, sumem 1.
    """
    
    # Millor solució trobada. Inicialment té cota infinit
    best_bound = np.inf
    best_board = board
    
    # Guardem en una cua de prioritat els taulells.
    # Guardarem les variables:
    # 1. Distància mínima (cota inferior) entre el tauler actual i el tauler solució
    # 2. Número de passes que duem en aquest tauler, g(X)
    # 3. El tauler
    pq = PriorityQueue()
    pq.put((board.manhattan_distance(), 0, board))
    
    # Com que els estats poden repetir-se al llarg de l'exploració, guardarem en un 'set' tots els
    # estats visitats. Així evidem tornar a visitar estats.
    existent_states = set([board.get_state_id()])
    expanded = 0
    
    while not pq.empty():
        
        # Obtenim un nou element de la cua
        curr_bound, curr_steps, curr_board = pq.get()
        expanded += 1

        # Mirem tots els moviments valids que podem fer des d'aquest tauler
        for a_move in curr_board.allowed_moves():
            new_board = curr_board.move(a_move)
            new_steps = curr_steps + 1
            new_bound = new_steps + new_board.manhattan_distance() # g(X) + h(X)
        
            # Si és un estat solució i ens millora la cota, actualitzem.
            if new_board.state():
                if new_bound < best_bound:
                    best_bound = new_bound
                    best_board = new_board
            
            # En cas de que no sigui solució però ens millori la cota. 
            elif (new_bound < best_bound) and (new_board.get_state_id() not in existent_states):
                existent_states.add(new_board.get_state_id())
                pq.put((new_bound,new_steps,new_board))
                
    return best_bound, best_board, expanded
    

In [46]:
board = NPuzzle()
board.create_board(moves=100, n=3)
print("Tauler inicial:")
print(board)
distance, final_board, expanded = solve_puzzle(board)
print(f"Solucionat en {distance} passos")
print(f"Taulers expandits: {expanded}")
print("Tauler final:")
print(final_board)

Tauler inicial:
+-----------+
| 8 | 7 | 5 | 
+-----------+
| 4 |   | 3 | 
+-----------+
| 2 | 6 | 1 | 
+-----------+

Solucionat en 24 passos
Taulers expandits: 2240
Tauler final:
+-----------+
| 1 | 2 | 3 | 
+-----------+
| 4 | 5 | 6 | 
+-----------+
| 7 | 8 |   | 
+-----------+

